In [1]:
from PY_tweet.lists_file import *

tweets_list = loadJoblib('DATA_all_texts/only_4labels_textdata.joblib')

# Janomeで形態素解析

In [2]:
!pip install janome

###　字句解析器作成

In [3]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter

tokenizer = Tokenizer()
token_filters = [POSStopFilter(['記号','助詞','助動詞','動詞'])]
a = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

In [4]:
test_tokens = a.analyze(tweets_list)

### 各ツイートの分かち書き

In [5]:
wakati = {}
for idx, tweet in enumerate(tweets_list):
    tokens = a.analyze(tweet)
    wakati[idx] = ' '.join([t.surface for t in tokens])

In [6]:
# wakati

# 単語頻出頻度計算

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import random
vectorizer = CountVectorizer()

In [8]:
X = vectorizer.fit_transform(wakati[i] for i in wakati.keys())
bows = {}
for i, bow in enumerate(X.toarray()):
    bows[i] = bow

In [9]:
# bows

### get_feature_names()でBowのインデックスに対応する配列

In [10]:
WORDS = vectorizer.get_feature_names()
# print(WORDS)

In [11]:
n = 1
for w_idx, count in enumerate(bows[1]):
    if count >= n:
        print(' -{}\t{}: WORDS[{}]'.format(count, WORDS[w_idx], w_idx))

 -2	19: WORDS[438]
 -1	co: WORDS[5171]
 -2	covid: WORDS[5321]
 -1	cxzfpqmq: WORDS[5734]
 -1	godiva: WORDS[10239]
 -1	https: WORDS[11790]
 -1	これ: WORDS[35549]
 -1	ウイルス: WORDS[38494]
 -1	コロナ: WORDS[40352]
 -1	ゴディバ: WORDS[40650]
 -1	名称: WORDS[51151]
 -1	新型: WORDS[55593]
 -1	続出: WORDS[60322]
 -1	英語: WORDS[61109]


# TF-IDF計算

### TF

In [12]:
def calc_tf(t_idx, w_idx):
    """t_idx番目のtweetのインデックスw_idxのTF値を算出する"""
    word_count = bows[t_idx][w_idx]
    if word_count == 0:
        return 0.0
    sum_of_words = sum(bows[t_idx])
    return word_count/float(sum_of_words)

In [23]:
index = 0
print('# TF values of a tweet:')
print('-' * 30)
print('{}'.format(tweets_list[0]))
print('-'* 30)
sample_tfs = [calc_tf(index, w_idx) for w_idx, word in enumerate(WORDS)]
tfs_sorted = sorted(enumerate(sample_tfs), key=lambda x:x[1], reverse=True)
for i, tf in tfs_sorted[:20]:
    print("{}\t{}".format(WORDS[i], round(tf, 4)))

# TF values of a tweet:
------------------------------
更新：WHOは新型コロナウイルスの正式名称を「COVID-19」とすると発表しました
https://t.co/zFQKGGXpse
------------------------------
19	0.0769
co	0.0769
covid	0.0769
https	0.0769
who	0.0769
zfqkggxpse	0.0769
ウイルス	0.0769
コロナ	0.0769
名称	0.0769
新型	0.0769
更新	0.0769
正式	0.0769
発表	0.0769
00	0.0
000	0.0
0000	0.0
0000044	0.0
0001	0.0
001	0.0
003	0.0


### IDF 

In [14]:
import math

def calc_idf(w_idx):
    """WORD[w_idx] の IDF値を算出する"""
    N = len(bows) #総文書数
    #単語 word が出現する文書数 df を計算
    df = 0
    for i in range(N):
        if bows[i][w_idx] > 0:
            df += 1
#     print(f'{w_idx}番目の単語は{df}回出現')
    return math.log2(N/float(df + 1)) #idf を計算

In [15]:
IDFS = [calc_idf(w_idx) for w_idx, word in enumerate(WORDS)]

In [16]:
idfs_sorted  = sorted(enumerate(IDFS), key=lambda x:x[1], reverse=True)
print("# IDF values")
for w_idx, idf in idfs_sorted[:10]:
    print("{}\t{}".format(WORDS[w_idx], round(idf, 4)))
print("︙")
for w_idx, idf in idfs_sorted[-10:]:
    print("{}\t{}".format(WORDS[w_idx], round(idf, 4)))

# IDF values
0000044	14.1951
0039	14.1951
004	14.1951
0057	14.1951
0058	14.1951
007758	14.1951
008	14.1951
0083	14.1951
00848	14.1951
00952	14.1951
︙
19	4.0932
影響	4.0932
対策	3.6473
こと	3.4964
感染	2.5809
ウイルス	2.1314
新型	1.8401
https	0.8388
co	0.8388
コロナ	0.0677


### TF-IDFの計算

In [17]:
def calc_tfidf(t_idx, w_idx):
    """t_idx 番目のブログの WORD[w_idx] の TF-IDF値を算出する"""
    return calc_tf(t_idx, w_idx) * calc_idf(w_idx)

In [18]:
sample_tfidf = [calc_tfidf(index, w_idx) for w_idx, word in enumerate(WORDS)]

In [ ]:
index = 0
print('# TFIDF values of a tweet:')
print('-' * 30)
print('{}'.format(tweets_list[1]))
print('-'* 30)
sample_tfs = [calc_tf(index, w_idx) for w_idx, word in enumerate(WORDS)]
tfs_sorted = sorted(enumerate(sample_tfs), key=lambda x:x[1], reverse=True)
for i, tf in tfs_sorted[:20]:
    print("{}\t{}".format(WORDS[i], round(tf, 4)))

In [28]:
tweets_list[0]

'更新：WHOは新型コロナウイルスの正式名称を「COVID-19」とすると発表しました\nhttps://t.co/zFQKGGXpse'

In [21]:
tfidf_sorted = sorted(enumerate(sample_tfidf), key=lambda x:x[1], reverse=True)
print("# TF-IDF values")

for w_idx, idf in tfidf_sorted:
    print("{}\t{}".format(WORDS[w_idx], idf))
# print("︙")
# for w_idx, idf in idfs_sorted[-10:]:
#     print("{}\t{}".format(WORDS[w_idx], round(idf, 4)))

# TF-IDF values
zfqkggxpse	1.0919339600609053
名称	0.8161676138515856
正式	0.7842416523685977
who	0.5653578022959533
更新	0.4848736751979583
発表	0.4623281644525758
covid	0.32194714334028157
19	0.31485890844942593
ウイルス	0.16395456619960375
新型	0.14154351800943915
https	0.06452515952797287
co	0.06451986850594002
コロナ	0.005208563575534095
00	0.0
000	0.0
0000	0.0
0000044	0.0
0001	0.0
001	0.0
003	0.0
0039	0.0
004	0.0
0057	0.0
0058	0.0
006	0.0
007	0.0
007758	0.0
008	0.0
0083	0.0
00848	0.0
009	0.0
00952	0.0
01	0.0
010	0.0
011	0.0
0115	0.0
012	0.0
0120	0.0
0120607021	0.0
01221	0.0
0123456	0.0
013	0.0
014	0.0
0153	0.0
0157	0.0
017	0.0
018	0.0
02	0.0
020	0.0
02020	0.0
0206	0.0
021	0.0
02266	0.0
024	0.0
025	0.0
026	0.0
027	0.0
028	0.0
029	0.0
0299597669	0.0
03	0.0
030	0.0
0303	0.0
031	0.0
0311	0.0
0312	0.0
032	0.0
0327	0.0
03281105	0.0
0330	0.0
0331	0.0
035	0.0
036	0.0
037	0.0
0394	0.0
04	0.0
040	0.0
0410	0.0
0415	0.0
0417	0.0
042	0.0
0421	0.0
0432252207	0.0
044	0.0
045	0.0
0457	0.0
047	0.0
048	0.0
0492241